In [1]:
# Some necessary code
from collections import defaultdict, Counter
import re, datetime
from functools import partial
import os
from IPython.display import display
import sys

def tokenize(message):
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    return (all_words)                           # remove duplicates

# QUESTION 1
###  Top n words from list of document files. 

In [2]:
# Defining a generic map reduce function for question 2,3,4

def map_reduce(inputs, mapper, reducer,n=None,a=None):
    
    collector = defaultdict(list)
    
    # writing a for loop over the inputs that calls mapper
    for input in inputs:
        for key, value in mapper(input):
            collector[key].append(value)
    
    # return statement that calls the reducer       
    
    return [output
           for key, value in collector.items()
           for output in reducer(key,value,n,a)]        

In [3]:
# defining a mapper which will read files and yield word with a count 1 for all files

def file_mapper(filename):
    with open(filename, 'r') as f:
            for line in f: 
                for word in tokenize(line):
                    yield ('Top_n_words',(word,1))

# defining a reducer which will sum the counts for each word and return n words with highest counts

def sum_max_reducer(files,words_and_counts,n,a):
    word_counts=Counter()
    for word,count in words_and_counts:
        word_counts[word]+=count
        
# find n most common words and return that (key,value) pair    
    
    yield (files,word_counts.most_common(n))
    


In [4]:
filenames = ["dataset1/genesis.txt",
            "dataset1/Luke.txt",
            "dataset1/Kings.txt"]
word_counts = map_reduce(filenames, file_mapper, sum_max_reducer,6)
word_counts


[('Top_n_words',
  [('and', 7478),
   ('the', 6041),
   ('of', 3330),
   ('he', 1769),
   ('to', 1634),
   ('in', 1419)])]

In [5]:
# converting output list of tuples into pandas dataframe to make it visually better.
import pandas as pd
x=word_counts[0][1]
a=[]
b=[]
for i in x:
    a.append(i[0])
    b.append(i[1])

display(pd.DataFrame({'names':a,
             'counts':b},columns=['names','counts']))

,names,counts
0,and,7478
1,the,6041
2,of,3330
3,he,1769
4,to,1634
5,in,1419


# QUESTION 2
### 'n' most common names that start with a given letter from a list of files 

In [6]:
# defining a mapper which will read files and yield word with a count 1 for all files

def files_mapper(file):
    with open(file, 'r') as f:
        for line in f: 
            for word in tokenizes(line):
                yield ('Top_N_words',(word,1))
                
# redefining tokenize function to extract only name from files

def tokenizes(message):
    x,y,z=message.split(',')
    x = x.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", x)   # extract the words
    return (all_words)   

In [7]:
# defining a reducer which will sum the counts for each word and return n words with highest counts
# if starting letter of name is equal to our input character

def reducer_most_common_letter_start(files,words_and_counts,n,a):
    word_counts=Counter()
    for word,count in words_and_counts:
        if word[0]==a:
            word_counts[word]+=count
            
    if len(word_counts)>n:
        yield (files,word_counts.most_common(n))
    else:
        yield (files,word_counts.most_common)

In [8]:
arr_txt = [x for x in os.listdir() if x.endswith(".txt")]
word_counts = map_reduce(arr_txt, files_mapper, reducer_most_common_letter_start,12,'q')
print(word_counts)

[('Top_N_words', [('quincy', 238), ('quinn', 160), ('queen', 142), ('quentin', 141), ('queenie', 132), ('quinton', 118), ('quintin', 106), ('quincey', 105), ('quinten', 98), ('quin', 98), ('quenton', 93), ('quince', 76)])]


In [9]:
# converting output list of tuples into pandas dataframe to make it visually better.
import pandas as pd
x=word_counts[0][1]
a=[]
b=[]
for i in x:
    a.append(i[0])
    b.append(i[1])

display(pd.DataFrame({'names':a,
             'counts':b},columns=['names','counts']))

,names,counts
0,quincy,238
1,quinn,160
2,queen,142
3,quentin,141
4,queenie,132
5,quinton,118
6,quintin,106
7,quincey,105
8,quinten,98
9,quin,98


# QUESTION 3
### top 'n' names from a list of files that contain a given string anywhere within a name in a list of files

In [10]:
# defining a mapper which will read files and yield word with a count 1 for all files

def files_mapper(file):
    with open(file, 'r') as f:
        for line in f: 
            for word in tokenizes(line):
                yield ('Top_N_words',(word,1))

# defining a reducer which will sum the counts for each word and return n words with highest counts
# if our input string is present anywhere in the name

def letter_most_occuring(files,words_and_counts,n,a):
    word_counts=Counter()
    for word,count in words_and_counts:
        if a in word:
            word_counts[word]+=count
    if len(word_counts)>n:
        yield (files,word_counts.most_common(n))
    else:
        yield (files,word_counts.most_common)

In [11]:
word_counts = map_reduce(arr_txt, files_mapper, letter_most_occuring,10,'eth')
print(word_counts)

[('Top_N_words', [('elizabeth', 273), ('kenneth', 237), ('ethel', 217), ('bethel', 197), ('seth', 177), ('beth', 166), ('ethan', 148), ('elisabeth', 141), ('elizebeth', 137), ('letha', 135)])]


In [12]:
# converting output list of tuples into pandas dataframe to make it visually better.
import pandas as pd
x=word_counts[0][1]
a=[]
b=[]
for i in x:
    a.append(i[0])
    b.append(i[1])
pd.DataFrame({'names':a,
             'counts':b},columns=['names','counts'])

,names,counts
0,elizabeth,273
1,kenneth,237
2,ethel,217
3,bethel,197
4,seth,177
5,beth,166
6,ethan,148
7,elisabeth,141
8,elizebeth,137
9,letha,135
